In [14]:
from modelzipper.tutils import *
import re
from pprint import pprint
import itertools
import random
import transformers
from datasets import Dataset, concatenate_datasets, DatasetDict, load_from_disk, load_dataset
import pandas as pd
import numpy as np

检查预训练种子数据，从32K的预训练数据集里面加载，然后切分这个数据集

In [3]:
pretrained_data_path = '/data/zecheng/data/SlimPajama-6B-chunked-32768-tokenized-llama3'
pretrained_data = load_dataset(pretrained_data_path, cache_dir='/data/zecheng/cache', split='train', num_proc=48)
print('数据集长度为', len(pretrained_data))

model_path = '/data/zecheng/hf_models/Llama-3-8B-Instruct-80K-QLoRA-Merged'
tokenizer = transformers.AutoTokenizer.from_pretrained(model_path)

def map_fn(examples):
    decoded_text = tokenizer.decode(examples['input_ids'])
    return {'text': decoded_text}

str_pretrained_data = pretrained_data.map(map_fn, num_proc=48, remove_columns=['input_ids', 'attention_mask', 'sentence_start_positions', 'labels'])

数据集长度为 16000


Map (num_proc=24): 100%|██████████| 16000/16000 [01:36<00:00, 165.81 examples/s]


In [22]:
subset = auto_read_data('/data/zecheng/data/llama3-80k-train-data/long-llm/gpt/multi_detail_paper_short.train.json')

In [45]:
print(len(subset[0]['conversations']))
# pprint(subset[0]['conversations'][2])

subset[0]['conversations'][3::2]

10


[{'content': 'Each paper adopts a distinct methodology tailored to its specific problem domain. Paper 1 proposes a novel methodology for ontology evaluation based on the Goal, Question, Metric approach, incorporating knowledge representation roles and specific evaluation criteria. Paper 2 develops a controlled natural language, TEDEI, for ontology authoring, translating English sentences into OWL-DL axioms. Paper 3 utilizes a logic-like formal language, TOP, for capturing the semantics of English sentences involving time, translating them into TSQL2 queries for temporal databases. Paper 4, PyReason, is implemented based on generalized annotated logic, extending it with various practical features for reasoning over knowledge graphs, including temporal extensions and type checking.',
  'role': 'assistant'},
 {'content': "The main contributions of these papers, while distinct, revolve around enhancing logic-based systems for specific applications. Paper 1 contributes a step-by-step approa

#### 检查关键路径数据

In [25]:
DATA_DIR = '/data/zecheng/data/llama3-80k-train-data/long-llm-filtered'
critical_data_names = ['bio_book.jsonl', 'multi_detail_paper_long.jsonl', 'one_detail_book.jsonl', 'multi_detail_book.jsonl', 'multi_detail_paper_short.jsonl',  'one_detail_paper.jsonl']
critical_data_paths = [os.path.join(DATA_DIR, name) for name in critical_data_names]

# bio_book.jsonl
bio_book_data = auto_read_data(critical_data_paths[0])
print(len(bio_book_data))
print(len(bio_book_data[4]['context']))
# pprint(bio_book_data[2])

# one_detail_paper.jsonl
one_detail_paper = auto_read_data(critical_data_paths[-1])
print(len(one_detail_paper))
print(len(one_detail_paper[4]['context']))
print(one_detail_paper[4].keys())

print(one_detail_paper[4]['context'])

224
12
32571
12
dict_keys(['question', 'answer', 'context'])
['loading scheme, respectively. In addition, from the resultant expressions, we can know the rate coverage probabilities of the macro-users $\\mathcal{\\bar{R}}_{U=0,1}\\left(\\tau\\right)$, the unoffloaded pico-users $\\mathcal{\\bar{R}}_{U=0,2\\bar{O}}(\\tau)$ and the offloaded users $\\mathcal{\\bar{R}}_{U=0,2O}\\left(\\tau\\right)$, where $\\mathcal{\\bar{R}}_{U=0,k}(\\tau)=\\mathcal{\\bar{R}}_{{\\rm IN},k}(0,\\tau)$ $\\left(k\\in\\{1,2O\\}\\right)$ and $\\mathcal{\\bar{R}}_{U=0,2\\bar{O}}(\\tau)=\\mathcal{R}_{{\\rm IN},2\\bar{O}}(\\tau)$. Here, we omit the expressions of the rate coverage probability and its MLA of the simple offloading scheme. Note that \\cite{gupta13} also derived the rate coverage probability and its MLA of the macro-users and the pico-users under the simple offloading scheme in large multi-antenna HetNets. However, they did not further obtain the results for the unoffloaded pico-users and the offload

In [24]:
print(one_detail_paper[4]['question'])

How is the resource divided in ABS?


In [3]:
CHUNK_NUM = 64

def extract_qa_pairs(text):
    sp_text = text.split("####")
    if len(sp_text) < 9:
        return None
    q1, a1, q2, a2 = sp_text[2].strip(), sp_text[4].strip(), sp_text[6].strip(), sp_text[8].strip()
    # pattern = r"####Question \d+####(.*?)####Answer \d+####(.*?)(?=####Question \d+####|$)"
    # matches = re.findall(pattern, text, re.DOTALL)
    
    res = [{"question": q1, "answer": a1}, {"question": q2, "answer": a2}]
    # for i, match in enumerate(matches, 1):
    #     question_text, answer_text = match
    #     res.append({"question": question_text, "answer": answer_text})
    return res

def combine_data(data, chunk_num):
    combined_data = []
    for i in range(0, len(data), chunk_num):
        combined_data.append(data[i:i+chunk_num])
    return combined_data

def filter_func(sample):
    filtered_response = sample["filtered_response"]
    if all(response.lower() == "null" for response in filtered_response):
        return False
    if any(response.lower() == "yes" for response in filtered_response):
        return True
    return False

ana_data = [auto_read_data(f, mute=True) for f in auto_read_dir("/vepfs/wcf/G/zecheng/data/SlimPajama-6B/filtering_deepseek", file_suffix=".jsonl")]
ana_data = list(itertools.chain(*ana_data))
ana_data_df = pd.DataFrame(ana_data)

print(ana_data_df.head())
ana_data_hf = Dataset.from_pandas(ana_data_df)

filter_data_hf = ana_data_hf.filter(filter_func, num_proc=12)
filter_data_hf = filter_data_hf.remove_columns(["filtered_response", "id"])

number of files with prefix '' and suffix '.jsonl': 10
                                           reference  \
0  ed 200mm pilot line integration facility in th...   
1  for differing customer recipes and requirement...   
2  ications Laboratories. Together, these two mul...   
3  llion in order to meet the exploding number of...   
4   maximize yields, and streamline equipment set...   

                                            qa_pairs     id filtered_response  
0  [{'question': 'What is EPIC?', 'answer': 'EPIC...  31188        [YES, YES]  
1  [{'question': 'What challenge did Applied Mate...  31189        [YES, YES]  
2  [{'question': 'What does Applied Materials' in...  31190        [YES, YES]  
3  [{'question': 'What is the significance of 300...  31191        [YES, YES]  
4  [{'question': 'What is one way Applied Materia...  31192        [YES, YES]  


Filter (num_proc=12): 100%|██████████| 34616/34616 [00:03<00:00, 9805.78 examples/s] 


'\nfiles = auto_read_dir("/vepfs/wcf/G/zecheng/data/SlimPajama-6B", file_prefix="generated_QA_pairs_thread", file_suffix=".jsonl")\nall_data = [auto_read_data(file) for file in files]\nall_data = [item for sublist in all_data for item in sublist]\n\nprint(f"total pairs: {len(all_data)}, can be deviced into {len(all_data) // 4} instances")\n\nprocessed_data = []\nfor item in all_data:\n    qa_pairs = extract_qa_pairs(item[\'QA_pairs\'])\n    if qa_pairs is not None:\n        ref = item[\'reference\']\n        ref = ref if isinstance(ref, str) else ref[1] \n        processed_data.append({"reference": ref, "qa_pairs": qa_pairs})\n\n        \ntinyllama = transformers.AutoModelForCausalLM.from_pretrained("/vepfs/wcf/G/zecheng/hf_models/TinyLlama_v1.1").to(\'cuda:6\')\ntokenizer = transformers.AutoTokenizer.from_pretrained("/vepfs/wcf/G/zecheng/hf_models/TinyLlama_v1.1")\n'

In [4]:
filter_data_hf.save_to_disk("/vepfs/wcf/G/zecheng/data/SlimPajama-6B/filtering_deepseek/filtered_wo_ana")
for i in range(200):
    print("--------")
    print(filter_data_hf[i]['reference'])

Saving the dataset (1/1 shards): 100%|██████████| 31932/31932 [00:00<00:00, 156790.54 examples/s]


In [15]:
filter_data_hf[2]

{'reference': 'ications Laboratories. Together, these two multimillion dollar facilities give new depth to Applied Materials\' process integration capabilities and signify an ever expanding commitment to meet the technology needs of its customers.\nFabricating The Next Trillion Chips\nAt Applied Materials, we developed the system\'s chipmakers needed to fabricate more powerful and affordable chips. Today we are forging closer alliances with our customers to help them streamline their productions processes and lower operating costs. To do that we are developing innovative "smart manufacturing technologies that will enable fab to achieve much higher levels of efficiency.\nIn less than a decade, the number of semiconductor chips produced each year has more than doubled, growing from 176 billion in 1994 to over 373 billion in 2000. Even with the economic downturn that began in 2001, chips are still being manufactured at a rate of close to one billion per day. By 2010, the annual demand for

In [ ]:
from modelzipper.tutils import *
data = auto_read_data('/data/zecheng/data/llama3-80k-train-data/long-llm/longalpaca/train.json')

In [10]:
item = data[2]['conversations']
all_splits = item[0]['content'].split('\n')
context, all_questions = '\n'.join(all_splits[2:-2]).strip(), [all_splits[-1]]
all_answers = [item[1]['content']]

In [16]:
all_answers

['Some possible directions for future work are:\n\n1. Applying TAT to more complex architectures beyond convolutional networks. The paper showed that TAT can enable training of vanilla (shortcut-free) convolutional networks, but TAT may also be applicable to other architectures liketransformers  and GANs. Exploring these applications of TAT could discover new insights.\n\n2. Investigating the use of TAT as a tool for network interpretability. Since TAT allows training of networks without shortcuts and normalization layers, the hierarchical feature representations learned by such networks may be more interpretable. Studying this could provide a new perspective on network interpretability.  \n\n3. Combining TAT with trainable activation functions like PReLU. Although the paper found that initializing PReLU with TAT works better than random initialization, jointly optimizing the activation slopes during training may provide further benefits. Exploring this combination could yield even bet

In [26]:
critical_data_names = ['gpt-bio_book', 'gpt-multi_detail_paper_short', 'gpt-multi_detail_paper_long']
preds = ['pred_w_full_paths', 'pred_w_half_paths', 'pred_wo_critical_paths']
for n in preds:
    for x in critical_data_names:
        tmp1 = datasets.load_from_disk(f'/nvme/zecheng/data/iclr2025/llama2-train-data/long-llm-pred/chunk_16_size_1024/{n}/{x}')
        print(n, len(tmp1))         
# tmp2 = datasets.load_from_disk('/nvme/zecheng/data/iclr2025/llama3-80k-train-data/long-llm-pred/pred_w_half_paths/gpt-bio_book')

# print(len(tmp1))

pred_w_full_paths 224
pred_w_full_paths 400
pred_w_full_paths 410
pred_w_half_paths 224
pred_w_half_paths 400
pred_w_half_paths 410
pred_wo_critical_paths 224
pred_wo_critical_paths 400
pred_wo_critical_paths 410


In [33]:
pred_ids = random.sample(range(0, len(tmp1)), 10)

for i in pred_ids:
    print(str(i) + '\n---------------')
    print(tmp1[i]['predict'])

95
---------------

77
---------------


[Answer: The method used in the paper is based on the study of the geometry of the moduli spaces of stable maps and the properties of the stable maps themselves. It involves the use of various techniques such as stratification, transversality, and index calculations. The paper also relies on existing results in the field of Gromov-Witten theory and the study of the geometry of symplectic manifolds.]
193
---------------
 to be the graph whose vertices are the monomial generators of $I$ and two generators are connected if their product is in $I$. The \emph{connectivity} of a vertex $v$ is the number of edges connected to $v$ in $\Gamma$. The \emph{connectivity} of a vertex $v$ is the number of edges connected to $v$ in $\Gamma$. The \emph{connectivity} of a vertex $v$ is the number of edges connected to $v$ in $\Gamma$. The \emph{connectivity} of a vertex $v$ is the number of edges connected to $v$ in $\Gamma$. The \emph{connectivity} of a vertex 

In [10]:
# data_path = '/data/zecheng/data/llama3-80k-train-data/long-llm-score/gpt-one_detail_paper'
data_path = '/nvme/zecheng/data/iclr2025/llama3-80k-train-data/dpo_data/v4_1'
data = datasets.load_from_disk(data_path)
print(len(data['train']))

948


In [29]:
print(data['train'][0]['chosen_position_ids'])
print(data['train'][0]['chosen_attention_mask'])
print((np.array(data['train'][0]['chosen_position_ids'])==0).sum())
print((np.array(data['train'][0]['chosen_attention_mask'])==0).sum())
print((np.array(data['train'][0]['chosen_input_ids'])==0).sum())
print(data['train'][1]['chosen_position_ids'] == data['train'][0]['chosen_position_ids'])
print(len(data['train'][45]['chosen_position_ids']))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 42119, 42120, 42121, 42122, 42123, 42124, 42125, 42126, 42127, 42128, 42129, 42130, 42131, 42132, 42133, 42134, 42135, 42136, 42137, 42138, 42139, 42140, 42141, 42142, 42143, 42144, 42145, 42146, 42147, 42148, 42149, 42150, 42151, 42152, 42153, 42154, 42155, 42156, 42157, 42158, 42159, 42160, 42161, 42162, 42163, 42164, 42165, 42166, 42167, 42168, 42169, 42170, 42171, 42172, 42173, 42174, 42175, 42176, 42177, 42178, 42179, 42180, 42181, 42182, 42183, 42184, 42185, 42186, 42187, 42188, 42189, 42190, 42191, 42192, 42193, 42194, 42195, 42196, 42197, 42198, 42199, 42200, 42201, 42202, 42203, 42204, 42205, 42206, 42207, 42208, 42209, 42210, 42211, 42212, 42213, 42214, 42215, 42216, 42217, 42218, 42219, 42220, 42221, 42222, 42223, 42224, 42225, 42226, 42227, 42228, 42229, 42230, 42231, 42232, 42233, 42234, 42235, 42236, 42237, 42238, 42239, 42240, 42241, 42242, 42243, 42244, 42245, 42246, 42247, 42248, 42249, 42250, 4

In [4]:
data

DatasetDict({
    train: Dataset({
        features: ['chosen_input_ids', 'chosen_attention_mask', 'chosen_position_ids', 'chosen_labels', 'reject_1_input_ids', 'reject_1_attention_mask', 'reject_1_position_ids', 'reject_1_labels', 'reject_2_input_ids', 'reject_2_attention_mask', 'reject_2_position_ids', 'reject_2_labels'],
        num_rows: 11669
    })
    test: Dataset({
        features: ['chosen_input_ids', 'chosen_attention_mask', 'chosen_position_ids', 'chosen_labels', 'reject_1_input_ids', 'reject_1_attention_mask', 'reject_1_position_ids', 'reject_1_labels', 'reject_2_input_ids', 'reject_2_attention_mask', 'reject_2_position_ids', 'reject_2_labels'],
        num_rows: 400
    })
})

In [10]:
cnt = 0
for item in data:
    if item['judge_scores'] >= 1:
        cnt += 1
print(cnt)

13352
